<a href="https://colab.research.google.com/github/saivenkatreddy29/Implementation-of-Hands-on-Large-Language-models-book/blob/main/Jayalamar_book_chapter_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

tomatoes = load_dataset('rotten_tomatoes')
train_data, test_data = tomatoes['train'],tomatoes['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [12]:
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 8530
})

# Supervised Classification

**Hugging Face Trainer**

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels = 2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [13]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
  "Tokenize the input data"
  return tokenizer(examples['text'],truncation = True)

tokenized_train = train_data.map(preprocess_function,batched = True)
tokenized_test = test_data.map(preprocess_function, batched = True)



Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [14]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [15]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
  logits,labels = eval_pred
  predictions = np.argmax(logits,axis = -1)
  load_f1 = evaluate.load('f1')
  f1 = load_f1.compute(predictions = predictions, references = labels)['f1']
  return {'f1':f1}


In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir = 'model',
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    num_train_epochs = 1,
    weight_decay = 0.01,
    save_strategy = 'epoch',
    report_to = "none"
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics

)

<ipython-input-16-853703f8506a>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=267, training_loss=0.40972143166074143, metrics={'train_runtime': 4509.4309, 'train_samples_per_second': 1.892, 'train_steps_per_second': 0.059, 'total_flos': 246561781088880.0, 'train_loss': 0.40972143166074143, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3604084849357605,
 'eval_f1': 0.8515037593984962,
 'eval_runtime': 215.0432,
 'eval_samples_per_second': 4.957,
 'eval_steps_per_second': 0.158,
 'epoch': 1.0}

Freeze Layers

In [3]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

model_id = "bert-base-cased"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels = 2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [7]:
for name,param in model.named_parameters():
  print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [9]:
for name,param in model.named_parameters():

  if name.startswith('classifier'):
    param.requires_grad = True
  else:
    param.requires_grad = False


In [10]:
for name,param in model.named_parameters():
  print(f'pram:{name}--- {param.requires_grad}')

pram:bert.embeddings.word_embeddings.weight--- False
pram:bert.embeddings.position_embeddings.weight--- False
pram:bert.embeddings.token_type_embeddings.weight--- False
pram:bert.embeddings.LayerNorm.weight--- False
pram:bert.embeddings.LayerNorm.bias--- False
pram:bert.encoder.layer.0.attention.self.query.weight--- False
pram:bert.encoder.layer.0.attention.self.query.bias--- False
pram:bert.encoder.layer.0.attention.self.key.weight--- False
pram:bert.encoder.layer.0.attention.self.key.bias--- False
pram:bert.encoder.layer.0.attention.self.value.weight--- False
pram:bert.encoder.layer.0.attention.self.value.bias--- False
pram:bert.encoder.layer.0.attention.output.dense.weight--- False
pram:bert.encoder.layer.0.attention.output.dense.bias--- False
pram:bert.encoder.layer.0.attention.output.LayerNorm.weight--- False
pram:bert.encoder.layer.0.attention.output.LayerNorm.bias--- False
pram:bert.encoder.layer.0.intermediate.dense.weight--- False
pram:bert.encoder.layer.0.intermediate.dense.b

In [19]:
from transformers import TrainingArguments,Trainer

args = TrainingArguments(
    output_dir = 'model',
    learning_rate = 2e-5,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    num_train_epochs = 1,
    weight_decay = 0.01,
    save_strategy = 'epoch',
    report_to = "none"
)

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

<ipython-input-19-40407c37a33d>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [20]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=267, training_loss=0.7168752834591526, metrics={'train_runtime': 1627.9189, 'train_samples_per_second': 5.24, 'train_steps_per_second': 0.164, 'total_flos': 246561781088880.0, 'train_loss': 0.7168752834591526, 'epoch': 1.0})

In [21]:
trainer.evaluate()

{'eval_loss': 0.6794946193695068,
 'eval_f1': 0.5893536121673004,
 'eval_runtime': 219.424,
 'eval_samples_per_second': 4.858,
 'eval_steps_per_second': 0.155,
 'epoch': 1.0}

Freeze blocsk 1-5

In [26]:
model_id = 'bert-base-cased'
model = AutoModelForSequenceClassification.from_pretrained(model_id,num_labels = 2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

for index,(name,param) in enumerate(model.named_parameters()):
  if index <165:
    param.requires_grad = False



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
for index, (name, param) in enumerate(model.named_parameters()):
     print(f"Parameter: {index}{name} ----- {param.requires_grad}")

Parameter: 0bert.embeddings.word_embeddings.weight ----- False
Parameter: 1bert.embeddings.position_embeddings.weight ----- False
Parameter: 2bert.embeddings.token_type_embeddings.weight ----- False
Parameter: 3bert.embeddings.LayerNorm.weight ----- False
Parameter: 4bert.embeddings.LayerNorm.bias ----- False
Parameter: 5bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: 6bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: 7bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: 8bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: 9bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: 10bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: 11bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: 12bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: 13bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: 14bert.encoder.laye

In [29]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()


<ipython-input-29-ff98988a5cbf>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


TrainOutput(global_step=267, training_loss=0.5087790042720037, metrics={'train_runtime': 2090.3106, 'train_samples_per_second': 4.081, 'train_steps_per_second': 0.128, 'total_flos': 246561781088880.0, 'train_loss': 0.5087790042720037, 'epoch': 1.0})

In [30]:
trainer.evaluate()

{'eval_loss': 0.4017946720123291,
 'eval_f1': 0.8086710650329878,
 'eval_runtime': 200.9193,
 'eval_samples_per_second': 5.306,
 'eval_steps_per_second': 0.169,
 'epoch': 1.0}

Few Shot Classification

In [32]:
!pip install setfit -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 1.9 MB/s eta 0:00:00


In [34]:
from setfit import sample_dataset

sampled_train_data = sample_dataset(tomatoes['train'],num_samples = 16)

In [36]:
from setfit import SetFitModel
model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [47]:
from setfit import TrainingArguments as SetFitTrainingArguments
from setfit import Trainer as SetFitTrainer

args = SetFitTrainingArguments(
    num_epochs = 3,
    num_iterations = 20
)

args.evaluation_strategy = args.evaluation_strategy  # Fix: Changed eval_stratagey to evaluation_strategy

trainer = SetFitTrainer(
    model = model,
    args = args,
    train_dataset = sampled_train_data,
    eval_dataset = test_data,
    metric = 'f1'
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sentence_transformers/trainer.py:205: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `BCSentenceTransformersTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


AttributeError: 'CallbackHandler' object has no attribute 'tokenizer'